In [1]:
from bs4 import BeautifulSoup
import logging
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

In [21]:
link_template = 'https://dblp.org/db/conf/wsdm/wsdm2024.html'
def get_title_author(link):
    resp = requests.get(link, proxies=None)
    soup = BeautifulSoup(resp.content, 'html.parser')
    uls = soup.find_all('ul', class_='publ-list')
    titles, authors = [], []
    for ul in uls[2:]:  # 遍历所有 ul
        articles = ul.find_all('li', class_='entry inproceedings')  # 找到当前 ul 下的所有 li
        print(f"Found {len(articles)} articles in one ul.")
        for li in articles:
            title = li.find('span', class_='title').text
            author_list = [a.text for a in li.find_all('span', itemprop='name')]

            titles.append(title)
            authors.append(", ".join(author_list[:-1]))

    return titles, authors

In [22]:
titles, authors = get_title_author(link_template)
print(titles, authors)
print(titles[0], authors[0])

Found 109 articles in one ul.
Found 17 articles in one ul.
Found 7 articles in one ul.
Found 9 articles in one ul.
Found 16 articles in one ul.
Found 6 articles in one ul.
Found 6 articles in one ul.
['Let the LLMs Talk: Simulating Human-to-Human Conversational QA via Zero-Shot LLM-to-LLM Interactions.', 'IDoFew: Intermediate Training Using Dual-Clustering in Language Models for Few Labels Text Classification.', 'LabelCraft: Empowering Short Video Recommendations with Automated Label Crafting.', 'MAD: Multi-Scale Anomaly Detection in Link Streams.', 'Ranking with Long-Term Constraints.', 'LMBot: Distilling Graph Knowledge into Language Model for Graph-less Deployment in Twitter Bot Detection.', 'To Copy, or not to Copy; That is a Critical Issue of the Output Softmax Layer in Neural Sequential Recommenders.', 'PEFA: Parameter-Free Adapters for Large-scale Embedding-based Retrieval Models.', 'Empathetic Response Generation with Relation-aware Commonsense Knowledge.', 'Professional Networ

In [23]:
print(len(titles), len(authors))


170 170


In [24]:
def graph_mask(title):
    t = title.lower()
    conditions = ['graph neural networks', 'gnns', 'gnn', 'graph', 'graph learning', 'graph embedding', 'network embedding']
    return np.any([c in t for c in conditions])

In [25]:
with open(f'2024-WSDM.txt', 'w', encoding='utf8') as fw:
    for t, n in zip(titles, authors):
        if graph_mask(t):
            fw.write(f'1. **{t}**\n\n')
            fw.write(f'    *{n}*\n\n')